このノートブックでは、小さなデータを用いて、PRS の計算手順を説明します。実際のデータセットを用いる時の計算手順とは少し違いますが、まずは小さなデータセットを用いて、`jupyter notebook`，`python`，`hail` などに慣れてもらえたらと思います。

### Step 1 `hail` など、必要なモジュールを読み込みます

下記のコードを実行してください。
ページ上側のメニューバーにある `実行` ボタンを押下することで、実行することができます。

In [1]:
import hail as hl
hl.init()
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Running on Apache Spark version 3.1.1
SparkUI available at http://172.20.10.3:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.67-a673309b0445
LOGGING: writing to /Users/hacchy/prs-on-hail/hail-20210529-1541-0.2.67-a673309b0445.log


Loading BokehJS ...

### Step 2 ゲノムデータを読み込みます

下記のコードでは、一般的なゲノムデータのファイル形式である `VCF` フォーマット（`genome-data/1kg.JPT.chr22.dose.vcf.bgz`）から `hail` のファイル形式である `matrix table` フォーマット (`genome-data/1kg.JPT.chr22.mt`) に変換します。

下記のコードを実行した後、1分程度の待ち時間が生じます。

In [2]:
hl.import_vcf('genome-data/1kg.JPT.chr22.dose.vcf.bgz').write('genome-data/1kg.JPT.chr22.mt', overwrite=True)

2021-05-29 15:41:49 Hail: INFO: Coerced almost-sorted dataset
2021-05-29 15:42:34 Hail: INFO: wrote matrix table with 652193 rows and 104 columns in 1 partition to genome-data/1kg.JPT.chr22.mt
    Total size: 48.10 MiB
    * Rows/entries: 48.10 MiB
    * Columns: 858.00 B
    * Globals: 11.00 B
    * Smallest partition: 652193 rows (48.10 MiB)
    * Largest partition:  652193 rows (48.10 MiB)


`mt` というオブジェクトにゲノムデータを読み込みます。
なお、`mt` は `matrix table` の略です。

In [3]:
mt = hl.read_matrix_table('genome-data/1kg.JPT.chr22.mt')

読み込んだゲノムデータに含まれる研究対象者の人数とバリアントの個数を表示します。

In [4]:
mt.count()

(652193, 104)

```
(652193, 104)
```
と表示されました。

これは、次のことを意味します。
- 研究対象者の人数が 104 名
- バリアントの個数が 652,193個

### Step 3 ゲノムデータに `variantID` を追加します

下記のコードは、読み込まれたゲノムデータのバリアント情報を表示します。

`show(5)` は、先頭の 5 個のバリアントのみを表示する、という意味です。

In [5]:
mt.rows().show(5)

+---------------+------------+---------------+-----------+----------+----------+----------+----------+----------+
| locus         | alleles    | rsid          |      qual | filters  |  info.AF | info.MAF |  info.R2 | info.ER2 |
+---------------+------------+---------------+-----------+----------+----------+----------+----------+----------+
| locus<GRCh37> | array<str> | str           |   float64 | set<str> |  float64 |  float64 |  float64 |  float64 |
+---------------+------------+---------------+-----------+----------+----------+----------+----------+----------+
| 22:16050115   | ["G","A"]  | "22:16050115" | -1.00e+01 | {}       | 6.38e-03 | 6.38e-03 | 0.00e+00 |       NA |
| 22:16050213   | ["C","T"]  | "22:16050213" | -1.00e+01 | {}       | 7.57e-03 | 7.57e-03 | 0.00e+00 |       NA |
| 22:16050568   | ["C","A"]  | "22:16050568" | -1.00e+01 | {}       | 4.00e-04 | 4.00e-04 | 0.00e+00 |       NA |
| 22:16050607   | ["G","A"]  | "22:16050607" | -1.00e+01 | {}       | 1.00e-03 | 1.00e-03 | 0.00e+00 |       NA |
| 22:16050627   | ["G","T"]  | "22:16050627" | -1.00e+01 | {}       | 4.00e-04 | 4.00e-04 | 0.00e+00 |       NA |
+---------------+------------+---------------+-----------+----------+----------+----------+----------+----------+
showing top 5 rows

下記のコードは、ゲノムデータのバリアント情報に `variantID` を追加します。

In [6]:
mt = mt.annotate_rows(variantID = (hl.str(mt.locus.contig) + ":" + hl.str(mt.locus.position)) )

下記のコードは、`variantID`を追加した後のバリアント情報を表示します。

In [7]:
mt.rows().show(5)

+---------------+------------+---------------+-----------+----------+----------+----------+----------+----------+---------------+
| locus         | alleles    | rsid          |      qual | filters  |  info.AF | info.MAF |  info.R2 | info.ER2 | variantID     |
+---------------+------------+---------------+-----------+----------+----------+----------+----------+----------+---------------+
| locus<GRCh37> | array<str> | str           |   float64 | set<str> |  float64 |  float64 |  float64 |  float64 | str           |
+---------------+------------+---------------+-----------+----------+----------+----------+----------+----------+---------------+
| 22:16050115   | ["G","A"]  | "22:16050115" | -1.00e+01 | {}       | 6.38e-03 | 6.38e-03 | 0.00e+00 |       NA | "22:16050115" |
| 22:16050213   | ["C","T"]  | "22:16050213" | -1.00e+01 | {}       | 7.57e-03 | 7.57e-03 | 0.00e+00 |       NA | "22:16050213" |
| 22:16050568   | ["C","A"]  | "22:16050568" | -1.00e+01 | {}       | 4.00e-04 | 4.00e-04 | 0.00e+00 |       NA | "22:16050568" |
| 22:16050607   | ["G","A"]  | "22:16050607" | -1.00e+01 | {}       | 1.00e-03 | 1.00e-03 | 0.00e+00 |       NA | "22:16050607" |
| 22:16050627   | ["G","T"]  | "22:16050627" | -1.00e+01 | {}       | 4.00e-04 | 4.00e-04 | 0.00e+00 |       NA | "22:16050627" |
+---------------+------------+---------------+-----------+----------+----------+----------+----------+----------+---------------+
showing top 5 rows

`variantID`のカラムが追加されていることが分かります。

### Step 4 `imputation quality` に基づいてゲノムデータをフィルタリングします

下記のコードでは、各バリアントの `imputation quality` の分布を表示します。
1分程度の待ち時間が生じます。

In [8]:
p = hl.plot.histogram(mt.info.R2, title='Imputation Quality Histogram', legend='Imputation Quality (R2)')
show(p)

<img src="plots/small-data-plot-01.png" width=50% >

`imputation quality` が低い（R2 < 0.3）バリアントも多いことが分かります。

下記のコードは、`imputation quality` が低い（R2 < 0.3）バリアントを除外します。

In [9]:
mt_filt = mt.filter_rows(mt.info.R2>=0.3)

下記のコードは、`imputation quality` が低い（R2 < 0.3）バリアントを除外した後の分布を表示します。

In [10]:
p = hl.plot.histogram(mt_filt.info.R2, title='Imputation Quality Histogram', legend='Imputation Quality (R2)')
show(p)

<img src="plots/small-data-plot-02.png" width=50% >

`imputation quality` が低い（R2 < 0.3）バリアントが除外されたことが分かります。

下記のコードは、`imputation quality` が低い（R2 < 0.3）バリアントを除外したのちのバリアントの個数を表示します。

In [11]:
mt_filt.count()

(151403, 104)

```
(151403, 104)
```
と表示されました。

これは、次のことを意味します。
- 研究対象者の人数が 104 名
- バリアントの個数が 151,403個

下記のコードは、`imputation quality` が低い（R2 < 0.3）バリアントを除外した後の `minor allele frequency` の分布を表示します。

In [12]:
p = hl.plot.histogram(mt_filt.info.MAF, title='MAF Histogram', legend='MAF', bins=50)
show(p)

<img src="plots/small-data-plot-03.png" width=50% >

MAF<1% のバリアントが多くある一方、MAFが5%〜50%のバリアントはほぼ等しい個数あることが分かります。

### Step 5 PRSモデルを読み込みます

ここでは、PRSモデル `PGS000004` のみを読み込みます。

下記のコードを実行すると、`prs-models/PGS000004.txt`のデータが読み込まれます。

In [13]:
model_PGS000004 = hl.import_table('prs-models/PGS000004.txt', impute=True, force=True)

2021-05-29 15:43:28 Hail: INFO: Reading table to impute column types
2021-05-29 15:43:29 Hail: INFO: Finished type imputation
  Loading field 'chr_name' as type int32 (imputed)
  Loading field 'chr_position' as type int32 (imputed)
  Loading field 'effect_allele' as type str (imputed)
  Loading field 'reference_allele' as type str (imputed)
  Loading field 'effect_weight' as type float64 (imputed)
  Loading field 'allelefrequency_effect' as type float64 (imputed)


下記のコードは、PRSモデルに含まれるバリアントの個数を表示します。

In [14]:
model_PGS000004.count()

313

`313` 
と表示されました。

これは、PRSモデルに含まれるバリアントの個数が 313 個であることを意味します。

### Step 6 PRSモデルに `variantID` を追加します

下記のコードは、読み込んだ PRS モデルの最初の 5 行（5 個のバリアントの情報）を表示します。

In [15]:
model_PGS000004.show(5)

,,,,,
chr_name,chr_position,effect_allele,reference_allele,effect_weight,allelefrequency_effect
int32,int32,str,str,float64,float64
1,100880328,"""T""","""A""",3.73e-02,4.10e-01
1,10566215,"""G""","""A""",-5.86e-02,3.29e-01
1,110198129,"""C""","""CAAA""",4.58e-02,7.76e-01
1,114445880,"""A""","""G""",6.21e-02,1.66e-01
1,118141492,"""C""","""A""",4.52e-02,2.66e-01


下記のコードは、読み込んだ PRS モデルに `variantID` を追加します。

In [16]:
model_PGS000004 = model_PGS000004.annotate(
    variantID = hl.str(model_PGS000004.chr_name) + ":" + hl.str(model_PGS000004.chr_position) 
) 

`variantID` を追加したのちの最初の 5 行を表示します。

In [17]:
model_PGS000004.show(5)

,,,,,,
chr_name,chr_position,effect_allele,reference_allele,effect_weight,allelefrequency_effect,variantID
int32,int32,str,str,float64,float64,str
1,100880328,"""T""","""A""",3.73e-02,4.10e-01,"""1:100880328"""
1,10566215,"""G""","""A""",-5.86e-02,3.29e-01,"""1:10566215"""
1,110198129,"""C""","""CAAA""",4.58e-02,7.76e-01,"""1:110198129"""
1,114445880,"""A""","""G""",6.21e-02,1.66e-01,"""1:114445880"""
1,118141492,"""C""","""A""",4.52e-02,2.66e-01,"""1:118141492"""


`variantID`のカラムが追加されていることが分かります。

### Step 7 ゲノムデータとPRSモデルに共通するバリアントを抽出します

下記のコードは、PRSモデルのバリアント情報を `variantID` で検索できるようにします。

In [18]:
model_PGS000004 = model_PGS000004.key_by('variantID')

下記のコードは、ゲノムデータと PRS モデルに共通するバリアントを抽出します。

In [19]:
mt_match = mt_filt.annotate_rows(**model_PGS000004[mt_filt.variantID])
mt_match = mt_match.filter_rows(hl.is_defined(mt_match.effect_weight))

下記のコードは、ゲノムデータと PRS モデルに共通するバリアントを表示します。
1分程度の待ち時間が生じます。

In [20]:
mt_match.rows().show()

2021-05-29 15:43:47 Hail: INFO: Coerced sorted dataset
2021-05-29 15:43:47 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 15:43:57 Hail: INFO: Ordering unsorted dataset with network shuffle


+---------------+-----------------------+---------------+-----------+----------+----------+----------+----------+----------+
| locus         | alleles               | rsid          |      qual | filters  |  info.AF | info.MAF |  info.R2 | info.ER2 |
+---------------+-----------------------+---------------+-----------+----------+----------+----------+----------+----------+
| locus<GRCh37> | array<str>            | str           |   float64 | set<str> |  float64 |  float64 |  float64 |  float64 |
+---------------+-----------------------+---------------+-----------+----------+----------+----------+----------+----------+
| 22:19766137   | ["C","T"]             | "22:19766137" | -1.00e+01 | {}       | 6.29e-01 | 3.71e-01 | 9.83e-01 |       NA |
| 22:29135543   | ["G","A"]             | "22:29135543" | -1.00e+01 | {}       | 9.21e-02 | 9.21e-02 | 9.92e-01 |       NA |
| 22:38583315   | ["AAAAG","A"]         | "22:38583315" | -1.00e+01 | {}       | 5.71e-03 | 5.71e-03 | 8.29e-01 |       NA |
| 22:38583315   | ["AAAAG","AAAAGAAAG"] | "22:38583315" | -1.00e+01 | {}       | 2.38e-01 | 2.38e-01 | 9.85e-01 |       NA |
| 22:39343916   | ["T","A"]             | "22:39343916" | -1.00e+01 | {}       | 4.09e-01 | 4.09e-01 | 1.00e+00 |       NA |
| 22:40904707   | ["CT","C"]            | "22:40904707" | -1.00e+01 | {}       | 3.43e-01 | 3.43e-01 | 9.94e-01 |       NA |
| 22:45319953   | ["G","A"]             | "22:45319953" | -1.00e+01 | {}       | 2.69e-01 | 2.69e-01 | 1.00e+00 |       NA |
| 22:46283297   | ["G","A"]             | "22:46283297" | -1.00e+01 | {}       | 4.97e-02 | 4.97e-02 | 9.45e-01 |       NA |
+---------------+-----------------------+---------------+-----------+----------+----------+----------+----------+----------+

+---------------+----------+--------------+---------------+------------------+---------------+------------------------+
| variantID     | chr_name | chr_position | effect_allele | reference_allele | effect_weight | allelefrequency_effect |
+---------------+----------+--------------+---------------+------------------+---------------+------------------------+
| str           |    int32 |        int32 | str           | str              |       float64 |                float64 |
+---------------+----------+--------------+---------------+------------------+---------------+------------------------+
| "22:19766137" |       22 |     19766137 | "T"           | "C"              |     -3.67e-02 |               3.80e-01 |
| "22:29135543" |       22 |     29135543 | "A"           | "G"              |      6.54e-02 |               8.70e-02 |
| "22:38583315" |       22 |     38583315 | "AAAAGAAAG"   | "AAAAG"          |     -4.71e-02 |               2.81e-01 |
| "22:38583315" |       22 |     38583315 | "AAAAGAAAG"   | "AAAAG"          |     -4.71e-02 |               2.81e-01 |
| "22:39343916" |       22 |     39343916 | "A"           | "T"              |      4.07e-02 |               2.54e-01 |
| "22:40904707" |       22 |     40904707 | "C"           | "CT"             |      1.15e-01 |               1.10e-01 |
| "22:45319953" |       22 |     45319953 | "A"           | "G"              |     -1.34e-02 |               4.17e-01 |
| "22:46283297" |       22 |     46283297 | "A"           | "G"              |      7.36e-02 |               1.12e-01 |
+---------------+----------+--------------+---------------+------------------+---------------+------------------------+

ゲノムデータと PRS モデルに共通するバリアントを抽出するために、`variantID` を利用しています。
`variantID` は、`染色体番号` と `塩基ポジション` から構成されます。

`22`番染色体のポジション`38583315`（バリアントID `22:38583315`）のバリアントのように、multi-allelic なバリアントの場合は、複数行に跨って表示されます。
PRSを計算する際には、`染色体番号` と `塩基ポジション` だけでなく、`effect_allele` と `reference_allele` もマッチさせる必要があります。
（そのための手順は後ほど解説します）

下記のコードは、ゲノムデータとPRSモデルに共通するバリアントを抽出した結果、ユニークな `variantID` が何個あるかをカウントします。
1分程度の待ち時間が生じます。

In [21]:
len(dict(mt_match.aggregate_rows(hl.agg.counter(mt_match.variantID))))

2021-05-29 15:44:09 Hail: INFO: Coerced sorted dataset
2021-05-29 15:44:09 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 15:44:17 Hail: INFO: Ordering unsorted dataset with network shuffle


7

下記のコードは、PRSモデルのうち、22番染色体に位置するバリアントの個数を表示します。

In [22]:
model_PGS000004.filter(model_PGS000004.chr_name==22).count()

11

本チュートリアルでは、小さなデータとして、22番染色体のゲノムデータのみを用いています。
上記の結果から、PRSモデルの22番染色体バリアント 11 個のうち、7 個がゲノムデータに含まれていたことが分かります。

このケースでは、残る 4 個は `imputation quality` が低く (R2<0.3) 除外されたバリアントでした。

### Step 8 抽出したゲノムデータを保存します

今後のステップの実行時間を短縮するため、抽出したゲノムデータを保存し、再度読み込みます。

下記のコードは、抽出したゲノムデータを `genome-data/1kg.JPT.chr22.matched.mt` に保存します。

In [23]:
mt_match.write('genome-data/1kg.JPT.chr22.matched.mt', overwrite=True)

2021-05-29 15:44:35 Hail: INFO: Coerced sorted dataset
2021-05-29 15:44:35 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 15:44:42 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 15:44:47 Hail: INFO: wrote matrix table with 8 rows and 104 columns in 1 partition to genome-data/1kg.JPT.chr22.matched.mt
    Total size: 2.60 KiB
    * Rows/entries: 1.75 KiB
    * Columns: 858.00 B
    * Globals: 11.00 B
    * Smallest partition: 8 rows (1.75 KiB)
    * Largest partition:  8 rows (1.75 KiB)


下記のコードは、`genome-data/1kg.JPT.chr22.matched.mt` を読み込みます。

In [24]:
mt_match = hl.read_matrix_table('genome-data/1kg.JPT.chr22.matched.mt')

### Step 9 ゲノムデータとPRSモデルのアリル情報を照合します

下記のコードは、ゲノムデータのアリル情報とPRSモデルのアリル情報を比較し、合致しているかをチェックします。

ゲノムデータのアリル情報は、`mt_match.alleles[0]` と `mt_match.alleles[1]` に保存されています。
PRSモデルのアリル情報は、`mt_match.effect_allele` と `mt_match.reference_allele` に保存されています。

`mt_match.alleles[0]` と `mt_match.effect_allele` が一致しており、かつ、`mt_match.alleles[1]` と `mt_match.reference_allele` が一致している場合は、後ほど `allele flip` の操作が必要であるため、`flip` フラグを立てます。

`mt_match.alleles[1]` と `mt_match.effect_allele` が一致しており、かつ、`mt_match.alleles[0]` と `mt_match.reference_allele` が一致している場合は、後ほど `allele flip` の操作が必要ないため、`flip` フラグを立てません。

In [25]:
flip = hl.case().when( 
    (mt_match.effect_allele == mt_match.alleles[0]) 
    & (mt_match.reference_allele == mt_match.alleles[1]), True ).when( 
    (mt_match.effect_allele == mt_match.alleles[1])
    & (mt_match.reference_allele == mt_match.alleles[0]), False ).or_missing()

In [26]:
mt_match = mt_match.annotate_rows( flip=flip )

下記のコードは、`flip` フラグ情報を表示します。

In [27]:
mt_match.rows().show(5)

+---------------+-----------------------+---------------+-----------+----------+----------+----------+----------+----------+
| locus         | alleles               | rsid          |      qual | filters  |  info.AF | info.MAF |  info.R2 | info.ER2 |
+---------------+-----------------------+---------------+-----------+----------+----------+----------+----------+----------+
| locus<GRCh37> | array<str>            | str           |   float64 | set<str> |  float64 |  float64 |  float64 |  float64 |
+---------------+-----------------------+---------------+-----------+----------+----------+----------+----------+----------+
| 22:19766137   | ["C","T"]             | "22:19766137" | -1.00e+01 | {}       | 6.29e-01 | 3.71e-01 | 9.83e-01 |       NA |
| 22:29135543   | ["G","A"]             | "22:29135543" | -1.00e+01 | {}       | 9.21e-02 | 9.21e-02 | 9.92e-01 |       NA |
| 22:38583315   | ["AAAAG","A"]         | "22:38583315" | -1.00e+01 | {}       | 5.71e-03 | 5.71e-03 | 8.29e-01 |       NA |
| 22:38583315   | ["AAAAG","AAAAGAAAG"] | "22:38583315" | -1.00e+01 | {}       | 2.38e-01 | 2.38e-01 | 9.85e-01 |       NA |
| 22:39343916   | ["T","A"]             | "22:39343916" | -1.00e+01 | {}       | 4.09e-01 | 4.09e-01 | 1.00e+00 |       NA |
+---------------+-----------------------+---------------+-----------+----------+----------+----------+----------+----------+

+---------------+----------+--------------+---------------+------------------+---------------+------------------------+-------+
| variantID     | chr_name | chr_position | effect_allele | reference_allele | effect_weight | allelefrequency_effect |  flip |
+---------------+----------+--------------+---------------+------------------+---------------+------------------------+-------+
| str           |    int32 |        int32 | str           | str              |       float64 |                float64 |  bool |
+---------------+----------+--------------+---------------+------------------+---------------+------------------------+-------+
| "22:19766137" |       22 |     19766137 | "T"           | "C"              |     -3.67e-02 |               3.80e-01 | False |
| "22:29135543" |       22 |     29135543 | "A"           | "G"              |      6.54e-02 |               8.70e-02 | False |
| "22:38583315" |       22 |     38583315 | "AAAAGAAAG"   | "AAAAG"          |     -4.71e-02 |               2.81e-01 |    NA |
| "22:38583315" |       22 |     38583315 | "AAAAGAAAG"   | "AAAAG"          |     -4.71e-02 |               2.81e-01 | False |
| "22:39343916" |       22 |     39343916 | "A"           | "T"              |      4.07e-02 |               2.54e-01 | False |
+---------------+----------+--------------+---------------+------------------+---------------+------------------------+-------+
showing top 5 rows

- `flip`フラグが `False` の場合、`allele flip` の操作は必要ありません。
- `flip`フラグが `True` の場合、`allele flip` の操作が必要です。
- `flip`フラグが `NA` の場合、PRSモデルの `effect_allele` または `reference_allele` がゲノムデータのアリル（`alleles[0]`および`alleles[1]`）と合致していないため、PRS計算時には考慮しません。

### Step 10 PRSを計算します

下記のコードは、各々のバリアントについて研究対象者の持っているアリル数（`mt_match.DS`）とバリアントの重み（`mt_match.effect_weight`）を掛け合わせて、ゲノムデータとPRSモデルの共通するバリアントについて足し合わせます。
これにより、PRSを計算することができます。

ここで、研究対象者の持っているアリル数を、下記のように計算しています。
- `flip` フラグが `False` の場合、`allele flip` の操作は必要ないため、`mt_match.DS` の値をアリル数として用います。
- `flip` フラグが `True` の場合、`allele flip` の操作が必要なため、`2 - mt_match.DS` の値をアリル数として用います。

In [28]:
prs=hl.agg.sum( hl.float64( mt_match.effect_weight ) * 
                hl.if_else( mt_match.flip, 
                            2 - mt_match.DS,
                            mt_match.DS) )

In [74]:
mt_match = mt_match.annotate_cols(prs=prs)

下記のコードは、PRSの値を表示します。

In [75]:
mt_match.cols().show(5)

2021-05-29 15:30:46 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


,
s,prs
str,float64
"""TEST_NA18939_TEST_NA18939""",7.87e-02
"""TEST_NA18940_TEST_NA18940""",-7.93e-03
"""TEST_NA18941_TEST_NA18941""",3.10e-02
"""TEST_NA18942_TEST_NA18942""",-4.60e-02
"""TEST_NA18943_TEST_NA18943""",1.42e-01


下記のコードは、PRSの分布を表示します。

In [76]:
p = hl.plot.histogram(mt_match.prs, title="PRS Histogram", legend="PGS000004", bins=20)
show(p)

<img src="plots/small-data-plot-04.png" width=50% >

### Step 11 PRSの計算結果を保存します

下記のコードは、PRSの計算結果を `1kg.JPT.chr22.PGS000004.PRS.txt` ファイルに保存します。

In [77]:
mt_match.cols().export('1kg.JPT.chr22.PGS000004.PRS.txt')

2021-05-29 15:36:28 Hail: INFO: Coerced sorted dataset
2021-05-29 15:36:29 Hail: INFO: merging 16 files totalling 3.8K...
2021-05-29 15:36:29 Hail: INFO: while writing:
    1kg.JPT.chr22.PGS000004.PRS.txt
  merge time: 103.561ms


### 以上でこのチュートリアルは終了です